In [22]:
import lusid
import lusid.models as models
import os
from msrest.authentication import BasicTokenAuthentication

api_url = os.getenv("FBN_LUSID_API_URL")
api_token = {"access_token": os.environ["ACCESS_TOKEN"]}
credentials = BasicTokenAuthentication(api_token)
client = lusid.LUSIDAPI(credentials, api_url)

In [23]:
import uuid
from datetime import datetime
import pytz
from collections import namedtuple
        
def build_transaction(trade_spec):
    return models.TransactionRequest(
        transaction_id=str(uuid.uuid4()),
        type="StockIn",
        instrument_uid=trade_spec.id,
        settlement_currency="GBP",
        transaction_date=trade_spec.trade_date,
        settlement_date=trade_spec.trade_date,
        units=100,
        transaction_price=trade_spec.price,
        total_consideration=100 * trade_spec.price,
        source="Client")

In [24]:
# create the portfolio

scope = "finbourne"
guid = str(uuid.uuid4())
effective_date = datetime(2018, 1, 1, tzinfo=pytz.utc)

request = models.CreateTransactionPortfolioRequest(
    "portfolio-{0}".format(guid),
    "id-{0}".format(guid),
    base_currency="GBP",
    created=effective_date
)
result = client.create_portfolio(scope, request)
portfolio_id = result.id.code

In [25]:
# add initial batch of trades

TransactionSpec = namedtuple('TradeSpec', 'id price trade_date')
trade_specs = [
    TransactionSpec("FIGI_BBG000C6K6G9", 101, datetime(2018, 1, 1, tzinfo=pytz.utc)),
    TransactionSpec("FIGI_BBG000C04D57", 102, datetime(2018, 1, 2, tzinfo=pytz.utc)),
    TransactionSpec("FIGI_BBG000FV67Q4", 103, datetime(2018, 1, 3, tzinfo=pytz.utc))
]
trade_specs.sort(key=lambda ts: ts.id)

new_transactions = list(map(build_transaction, trade_specs))

# add initial batch of trades
initial_result = client.upsert_transactions(scope, portfolio_id, new_transactions)
as_at_batch1 = initial_result.version.as_at_date

In [26]:
# add trade for 2018-1-8
trade = build_transaction(TransactionSpec("FIGI_BBG000BF0KW3", 104, datetime(2018, 1, 8, tzinfo=pytz.utc)))
later_trade = client.upsert_transactions(scope, portfolio_id, [trade])

as_at_batch2 = later_trade.version.as_at_date

In [27]:
# add back dated trade
trade = build_transaction(TransactionSpec("FIGI_BBG000BF4KL1", 105, datetime(2018, 1, 5, tzinfo=pytz.utc)))
backdated_trade = client.upsert_transactions(scope, portfolio_id, [trade])

as_at_batch3 = backdated_trade.version.as_at_date

In [28]:
def print_transactions(transactions):
    for transaction in transactions:
        print("{0}\t{1}\t{2}\t{3}\t{4}".format(transaction.instrument_uid,
                                               transaction.transaction_date,
                                               transaction.units,
                                               transaction.transaction_price,
                                               transaction.total_consideration))


In [29]:
# get the list of original trades
trades_list = client.get_transactions(scope, portfolio_id, as_at=as_at_batch1)

assert(len(trades_list.values) == 3)
print("transactions at {0}".format(as_at_batch1))
print_transactions(trades_list.values)

transactions at 2018-09-26 17:14:32.950187+00:00
FIGI_BBG000C6K6G9	2018-01-01 00:00:00+00:00	100.0	101.0	10100.0
FIGI_BBG000C04D57	2018-01-02 00:00:00+00:00	100.0	102.0	10200.0
FIGI_BBG000FV67Q4	2018-01-03 00:00:00+00:00	100.0	103.0	10300.0


In [30]:
# get the list of trades including the later trade
trades_list = client.get_transactions(scope, portfolio_id, as_at=as_at_batch2)

assert(len(trades_list.values) == 4)
print("transactions at {0}".format(as_at_batch2))
print_transactions(trades_list.values)

transactions at 2018-09-26 17:14:33.547954+00:00
FIGI_BBG000C6K6G9	2018-01-01 00:00:00+00:00	100.0	101.0	10100.0
FIGI_BBG000C04D57	2018-01-02 00:00:00+00:00	100.0	102.0	10200.0
FIGI_BBG000FV67Q4	2018-01-03 00:00:00+00:00	100.0	103.0	10300.0
FIGI_BBG000BF0KW3	2018-01-08 00:00:00+00:00	100.0	104.0	10400.0


In [31]:
# get the list of trades including the back-dated trade
trades_list = client.get_transactions(scope, portfolio_id, as_at=as_at_batch3)

assert(len(trades_list.values) == 5)
print("transactions at {0}".format(as_at_batch3))
print_transactions(trades_list.values)

transactions at 2018-09-26 17:14:37.729743+00:00
FIGI_BBG000C6K6G9	2018-01-01 00:00:00+00:00	100.0	101.0	10100.0
FIGI_BBG000C04D57	2018-01-02 00:00:00+00:00	100.0	102.0	10200.0
FIGI_BBG000FV67Q4	2018-01-03 00:00:00+00:00	100.0	103.0	10300.0
FIGI_BBG000BF4KL1	2018-01-05 00:00:00+00:00	100.0	105.0	10500.0
FIGI_BBG000BF0KW3	2018-01-08 00:00:00+00:00	100.0	104.0	10400.0


In [32]:
# get the list of trades now
all_trades = client.get_transactions(scope, portfolio_id)
print("transactions at {0}".format(datetime.utcnow()))
print_transactions(all_trades.values)

transactions at 2018-09-26 17:14:43.368934
FIGI_BBG000C6K6G9	2018-01-01 00:00:00+00:00	100.0	101.0	10100.0
FIGI_BBG000C04D57	2018-01-02 00:00:00+00:00	100.0	102.0	10200.0
FIGI_BBG000FV67Q4	2018-01-03 00:00:00+00:00	100.0	103.0	10300.0
FIGI_BBG000BF4KL1	2018-01-05 00:00:00+00:00	100.0	105.0	10500.0
FIGI_BBG000BF0KW3	2018-01-08 00:00:00+00:00	100.0	104.0	10400.0
